In [1]:
import pandas as pd
import numpy as np
import rdkit
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, Descriptors, Lipinski
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator

# 1) Data preparation

## О данных:
Выборка примерно сбалансированная:
- 432 vs 361

In [2]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

In [3]:
data_path = "./data/small_dataset_solid_liquid.ssv"
columns = ['line_number', 'smiles', 'cas', 'label']

In [4]:
def create_df(data_path, columns) -> pd.DataFrame:
    df = pd.read_csv(data_path, sep=' ', header=None, names=columns)
    df["label"] = df['label'].replace({1: 0, 2: 1})
    return df

def compute_fingerprints(smiles, radius, nbits):
    mol = Chem.MolFromSmiles(smiles)
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nbits)
    return mfpgen.GetFingerprint(mol)

def compute_descriptors(smiles, descriptors):
    mol = Chem.MolFromSmiles(smiles)

    avail_descriptors = {
        'MolWt': lambda mol: Descriptors.MolWt(mol),
        'LogP': lambda mol: Descriptors.MolLogP(mol),
        'TPSA': lambda mol: Descriptors.TPSA(mol),
        'NumRotatableBonds': lambda mol: Descriptors.NumRotatableBonds(mol),
        'NumHDonors': lambda mol: Descriptors.NumHDonors(mol),
        'NumHAcceptors': lambda mol: Descriptors.NumHAcceptors(mol),
        'FractionCSP3': lambda mol: Lipinski.FractionCSP3(mol),
        'NumAromaticRings': lambda mol: Descriptors.NumAromaticRings(mol),
        'FractionRotatableBonds': lambda mol: rdMolDescriptors.CalcFractionCSP3(mol),

        # ----------
        'NumHBD': lambda mol: rdMolDescriptors.CalcNumHBD(mol),
        "NumHeavyAtoms": lambda mol: rdMolDescriptors.CalcNumHeavyAtoms(mol),
        # ----------
        
        'NumHBA': lambda mol: rdMolDescriptors.CalcNumHBA(mol),
        'NumRings': lambda mol: rdMolDescriptors.CalcNumRings(mol),
        'NumHeteroatoms': lambda mol: rdMolDescriptors.CalcNumHeteroatoms(mol),
        'Chi0v': lambda mol: rdMolDescriptors.CalcChi0v(mol),
        'Chi1v': lambda mol: rdMolDescriptors.CalcChi1v(mol),
        'Chi2v': lambda mol: rdMolDescriptors.CalcChi2v(mol),
    }

    X = []
    for desc_name in descriptors:
        X.append(avail_descriptors[desc_name](mol))
    
    return pd.Series(X)
    
def create_data(df, descriptors: list, apply_norm=False, radius=2, nbits=2048, fingerprints_pca=True, pca_dim=32) -> tuple:
    '''
    Arguments:
    -------
    radius: radius for Morgan fingerprints
    nbits: number of bits for Morgan fingerprints

    fingerprints_pca: use PCA on fingerprints features or not
    pca_dim: obvious

    Returns: (X_fps, X_at, y)
    ------
    X_fps: fingerprint features
    X_at: non-fingerprint atomic features
    y: label
    '''
    smiles = df['smiles']
    y = df['label'].values

    df['fingerprints'] = smiles.apply(compute_fingerprints, args=(radius, nbits,))
    df = df.dropna(subset=['fingerprints'])
    X_fps = np.array([np.array(fp) for fp in df['fingerprints']])
    if fingerprints_pca:
        pca = PCA(n_components=pca_dim)
        X_fps = pca.fit_transform(X_fps)

    X_at = smiles.apply(compute_descriptors, args=(descriptors,))
    X_at = np.array(X_at)

    if apply_norm:
        X_fps = normalize(X_fps)
        X_at = normalize(X_at)

    return X_fps, X_at, y

---
# 2) Model Creation & Learning

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score

In [6]:
def eval_metrics(y_true, y_pred):
    return {
        "ACC": accuracy_score(y_true, y_pred),
        "ROC-AUC": roc_auc_score(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }

In [7]:
descriptors = [
    "NumHBD",                # Number of Hydrogen Bond Donors (rdMolDescriptors)
    "NumHeavyAtoms",         # Number of Heavy Atoms

    'MolWt',                 # Molecular Weight
    'LogP',                  # LogP (octanol-water partition coefficient)
    'TPSA',                  # Topological Polar Surface Area
    'NumRotatableBonds',     # Number of Rotatable Bonds
    'NumHDonors',            # Number of Hydrogen Bond Donors
    'NumHAcceptors',         # Number of Hydrogen Bond Acceptors
    'FractionCSP3',          # Fraction of sp3 Hybridized Carbons
    'NumAromaticRings',      # Number of Aromatic Rings
    'FractionRotatableBonds',# Fraction of Rotatable Bonds
    'NumHBD',               
    'NumHBA',                # Number of Hydrogen Bond Acceptors (rdMolDescriptors)
    'NumRings',              # Number of Rings
    'NumHeteroatoms',        # Number of Heteroatoms
    'Chi0v',                 # Chi Connectivity Index 0 (Valence)
    'Chi1v',                 # Chi Connectivity Index 1 (Valence)
    'Chi2v',                 # Chi Connectivity Index 2 (Valence)
]

df = create_df(data_path, columns)

In [8]:
X_fps, X_at, y = create_data(
    df,
    descriptors=descriptors,
    apply_norm=False,
    radius=2,
    nbits=4096,
    fingerprints_pca=True,
    pca_dim=32,
)

X_combined = np.hstack([X_fps, X_at])

In [9]:
def fit_and_eval(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    metrics = eval_metrics(y_test, y_pred)

    return metrics

## 2.1) RandomForest on X_at

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

fit_and_eval(model, X=X_at, y=y)

{'ACC': 0.9245283018867925,
 'ROC-AUC': 0.9244535951853026,
 'R2': 0.69781438074121}

## 2.2) RandomForest on X_fps with PCA

In [12]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

fit_and_eval(model, X=X_fps, y=y)

{'ACC': 0.8238993710691824,
 'ROC-AUC': 0.8209534368070953,
 'R2': 0.29490022172949004}

## 2.3) RandomForst on [X_at, PCA(X_fps)]

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

fit_and_eval(model, X=X_combined, y=y)

{'ACC': 0.9371069182389937,
 'ROC-AUC': 0.9374406081723156,
 'R2': 0.7481786506176751}

---
## 2.4) Catboost on [X_at, PCA(X_fps)]

In [14]:
from catboost import CatBoostClassifier

**Current best**:
```python
data_args = {
    "descriptors": descriptors,
    "apply_norm": False,
    "radius": 2,
    "nbits": 4096,
    "fingerprints_pca": True,
    "pca_dim": 32,
}

model_args = {
    "iterations": 1000,
    "learning_rate": 0.1,
    "depth": 6,
    "loss_function": 'Logloss',
    "verbose": 0,
    "random_seed": 100
}
X = X_combined
```

In [26]:
data_args = {
    "descriptors": descriptors,
    "apply_norm": False,
    "radius": 2,
    "nbits": 4096,
    "fingerprints_pca": True,
    "pca_dim": 32,
}

model_args = {
    "iterations": 1000,
    "learning_rate": 0.1,
    "depth": 6,
    "loss_function": 'Logloss',
    "verbose": 0,
    "random_seed": 100
}

In [27]:
X_fps, X_at, y = create_data(df, **data_args)
X_combined = np.hstack([X_fps, X_at])
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [28]:
model = CatBoostClassifier(**model_args)
model.fit(X_train, y_train, eval_set=(X_test, y_test))

y_pred = model.predict(X_test)

In [29]:
eval_metrics(y_test, y_pred)

{'ACC': 0.9371069182389937,
 'ROC-AUC': 0.936648717136522,
 'R2': 0.7481786506176751}